In [1]:
import torch
from transformers import AutoTokenizer
from based.models.gpt import GPTLMHeadModel
import torch.nn as nn


/home/mocuto/anaconda3/envs/m2/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/mocuto/anaconda3/envs/m2/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/mocuto/anaconda3/envs/m2/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = GPTLMHeadModel.from_pretrained_hf("hazyresearch/based-360m")

No module named 'causal_attention_cuda'
Successfully imported the causal dot product kernel! 
Successfully imported the FLA triton kernels! 


In [3]:
model

GPTLMHeadModel(
  (transformer): GPTModel(
    (embeddings): GPT2Embeddings(
      (word_embeddings): Embedding(50264, 1024)
    )
    (layers): ModuleList(
      (0): Block(
        (mixer): BaseConv(
          (in_proj): Linear(in_features=1024, out_features=4096, bias=True)
          (out_proj): Linear(in_features=2048, out_features=1024, bias=True)
          (conv): ShortConvolution(
            (conv): Conv1d(2048, 2048, kernel_size=(3,), stride=(1,), padding=(2,), groups=2048, bias=False)
          )
        )
        (dropout1): Dropout(p=0, inplace=False)
        (drop_path1): StochasticDepth(p=0.0, mode=row)
        (norm1): RMSNorm()
        (mlp): GatedMlp(
          (fc1): Linear(in_features=1024, out_features=4096, bias=False)
          (fc2): Linear(in_features=2048, out_features=1024, bias=False)
        )
        (dropout2): Dropout(p=0, inplace=False)
        (drop_path2): StochasticDepth(p=0.0, mode=row)
        (norm2): RMSNorm()
      )
      (1): Block(
        (mi

In [4]:
model.cuda()

GPTLMHeadModel(
  (transformer): GPTModel(
    (embeddings): GPT2Embeddings(
      (word_embeddings): Embedding(50264, 1024)
    )
    (layers): ModuleList(
      (0): Block(
        (mixer): BaseConv(
          (in_proj): Linear(in_features=1024, out_features=4096, bias=True)
          (out_proj): Linear(in_features=2048, out_features=1024, bias=True)
          (conv): ShortConvolution(
            (conv): Conv1d(2048, 2048, kernel_size=(3,), stride=(1,), padding=(2,), groups=2048, bias=False)
          )
        )
        (dropout1): Dropout(p=0, inplace=False)
        (drop_path1): StochasticDepth(p=0.0, mode=row)
        (norm1): RMSNorm()
        (mlp): GatedMlp(
          (fc1): Linear(in_features=1024, out_features=4096, bias=False)
          (fc2): Linear(in_features=2048, out_features=1024, bias=False)
        )
        (dropout2): Dropout(p=0, inplace=False)
        (drop_path2): StochasticDepth(p=0.0, mode=row)
        (norm2): RMSNorm()
      )
      (1): Block(
        (mi

In [61]:
text = "Paris is the capital of "
input_ids = tokenizer.encode(text, return_tensors="pt")
output = model(input_ids.cuda(), num_last_tokens=2)[0]
prediction = torch.argmax(output, dim=-1)
prediction_text = tokenizer.decode(prediction[0], skip_special_tokens=True)
print(prediction_text)

 the the


In [62]:
input = tokenizer.encode("If I take one more step, it will be", return_tensors="pt").to("cuda")
output = model.generate(input, max_length=20)
print(tokenizer.decode(output[0]))

If I take one more step, it will be a very long time before I will be able to


In [63]:
input_ids = tokenizer.encode(text, return_tensors="pt")
output = model.transformer.forward(input_ids.cuda())
output

tensor([[[ 0.3466, -1.0931, -0.3796,  ...,  1.4669,  1.1784, -0.6306],
         [ 0.9851, -0.9446,  0.2412,  ...,  1.4911,  1.0009, -1.1894],
         [-0.3481,  0.6451, -0.4064,  ...,  1.4124,  0.5562, -1.0948],
         [ 1.2101, -2.3107,  0.3959,  ...,  1.3128, -0.5501, -1.4398],
         [ 0.4671, -0.2395,  0.8004,  ...,  2.0549,  0.0364,  0.5992],
         [ 0.6768, -0.1461,  0.9306,  ...,  1.3063,  0.4129,  0.0129]]],
       device='cuda:0', grad_fn=<DropoutAddLayerNormFnBackward>)

In [70]:
output2 = model.transformer.forward_with_hidden_states(output)
output2.shape

torch.Size([1, 6, 1024])

In [65]:
output3 = model.forward(input_ids, hidden_states=output2, num_last_tokens=1)
output3

CausalLMOutput(logits=tensor([[[ 1.4798,  5.1853, -0.6468,  ..., -8.5160, -8.8513, -8.6187]]],
       device='cuda:0', grad_fn=<UnsafeViewBackward0>))

In [66]:
prediction = torch.argmax(output3[0], dim=-1)
prediction_text = tokenizer.decode(prediction[0], skip_special_tokens=True)
print(prediction_text)

In [71]:
model.transformer.embeddings.word_embeddings.weight

Parameter containing:
tensor([[-0.0140,  0.1109, -0.0666,  ..., -0.0406,  0.0346, -0.0036],
        [-0.0723,  0.0188, -0.0229,  ...,  0.0148, -0.1590,  0.0202],
        [-0.0382, -0.2077,  0.0858,  ..., -0.0919,  0.0741,  0.0711],
        ...,
        [ 0.0222, -0.0407, -0.0543,  ...,  0.0659,  0.0081,  0.0398],
        [ 0.0137, -0.0373, -0.0338,  ...,  0.0561, -0.0003,  0.0436],
        [ 0.0155, -0.0379, -0.0462,  ...,  0.0733,  0.0429,  0.0510]],
       device='cuda:0', requires_grad=True)

In [69]:
model.lm_head

Linear(in_features=1024, out_features=50264, bias=False)